In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_train=pd.read_csv('../input/train.csv')
df_test_or=pd.read_csv('../input/test.csv')


In [ ]:
sns.stripplot(x='Utilities',y='SalePrice',data=df_train)

In [ ]:
df_train.describe()

In [ ]:
sns.distplot(df_train['SalePrice'])

In [ ]:
df_train.plot.scatter(x='GrLivArea', y='SalePrice', ylim=(0,800000));
df_train.plot.scatter(x='TotalBsmtSF', y='SalePrice', ylim=(0,800000));

In [ ]:
sns.stripplot(x='OverallQual',y='SalePrice',data=df_train)

In [ ]:
sns.stripplot(x='YearBuilt',y='SalePrice',data=df_train)

In [ ]:
#correlation matrix
corrmat=df_train.corr()
f, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(corrmat,vmax=.6,square=True)


In [ ]:
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols],size=2.5)
plt.show()


In [ ]:
total=df_train.isnull().sum().sort_values(ascending=False)
percent=(df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data=pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_data.head(20)



In [ ]:
df_train=df_train.drop((missing_data[missing_data['Total']>1]).index,1)
df_train=df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max()



In [ ]:
df_train.select_dtypes(include=['object'])

In [ ]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()



In [ ]:
idx=(df_train.select_dtypes(include=['object']).columns)
idxlist=idx.tolist()
for x in idxlist:
    df_train[x]=le.fit_transform(df_train[x])


In [ ]:
df_train.head(20)

In [ ]:
from sklearn.model_selection import train_test_split
data=df_train.iloc[:,:-1]
#data=df_train.loc[:,['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']]
labels=df_train.iloc[:,-1]
train_data,test_data,train_labels,test_labels=train_test_split(data,labels,random_state=2,train_size=0.7)

In [ ]:
print(data.head())

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg=RandomForestRegressor()
reg.fit(train_data,train_labels)

In [ ]:
predictions=reg.predict(test_data)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(test_labels,predictions))



In [ ]:
total2=df_test_or.isnull().sum().sort_values(ascending=False)
percent2=(df_test_or.isnull().sum()/df_test_or.isnull().count()).sort_values(ascending=False)
missing_data2=pd.concat([total2,percent2],axis=1,keys=['Total2','Percent2'])
missing_data2.head(25)


In [ ]:
df_test=pd.DataFrame(data=df_test_or,columns=train_data.columns.tolist())
df_test

In [ ]:
tot=df_test.select_dtypes(include=['float']).isnull().sum().sort_values(ascending=False)
strnull=df_test.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
#df_test[(tot).index.where(tot>0)]
#tot.index.where(tot>0)
df_test[tot.index]=df_test[tot.index].fillna(0)
df_test[strnull.index]=df_test[strnull.index].fillna('zzz')

In [ ]:
df_test.isnull().sum().sort_values(ascending=False)

In [ ]:
idx2=df_test.select_dtypes(include=['object']).columns
idxlist2=idx2.tolist()
for x in idxlist2:
    df_test[x]=le.fit_transform(df_test[x])

In [ ]:
prices=reg.predict(df_test)

sub=pd.concat([df_test_or['Id'],pd.Series(prices)],axis=1,keys=['Id','SalePrice'])
sub.to_csv('sub2.csv',index=False,header=True)

